In [11]:
import json

def preprocess_webnlg(path):
    with open(path,"r") as file:
        data=file.read()
    obj=json.loads(data)
    entries=obj["entries"]
    output_list=list()
    for i in range(128):#len(entries)):
        for lex in list(map(lambda el: el["lex"],entries[i][str((i+1))]["lexicalisations"])):
            output_list.append({
                "instruction": "The input is a list of triplets. Each triple has a subject, a predicate and an object. Try to produce a sentence out of it.",
                "input":entries[i][str((i+1))]["modifiedtripleset"],
                "original":"",
                "output": lex
            })
    with open(path[:-5]+"_cleaned.json", "w") as file:
        json.dump([ob for ob in output_list], file)

In [12]:
dataset_base_path="webnlg-dataset-master/"
dataset_version_path="release_v2.1/json/"
preprocess_webnlg(dataset_base_path+dataset_version_path+"webnlg_release_v2.1_train.json")
preprocess_webnlg(dataset_base_path+dataset_version_path+"webnlg_release_v2.1_dev.json")
preprocess_webnlg(dataset_base_path+dataset_version_path+"webnlg_release_v2.1_test.json")


In [13]:
from datasets import load_dataset
a=load_dataset("json",data_files="./webnlg-dataset-master/release_v2.1/json/webnlg_release_v2.1_dev_cleaned.json")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 701.62it/s]


Dataset json downloaded and prepared to /home/bagnol/.cache/huggingface/datasets/json/default-348d710258f8e114/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 631.29it/s]


In [14]:
a=a["train"]